Exploratory Data Analysis of "vehicles_us.csv"

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math as mt
from scipy import stats
import streamlit as st
import plotly.express as px
import altair as alt

print("Helloworldooss hit")

In [ ]:
#import csv file of vehicles information
vehicles = pd.read_csv("/Users/jking/Data_Science/Sprint4Project/vehicles_us.csv")

#vehicles.head()
vehicles.info()
vehicles.sample(6)

Based on what we see using .info() and .sample(), the columns('model_year', 'cylinders', 'odometer','paint_color', 'is_4wd') have missing values. We can also see that the columns('model_year', 'cylinders', 'odometer', 'is_4wd') are pointlessly using datatype float64, which takes up unecesary memory. I will go through each one of these columns to see what should been done about this missing values. I will also convert these float64 columns to int64. Finally I wll convert the 'date_posted' column to date time for convenience. 

In [ ]:
#function for quick analysis of missing values within column
def quick_column(column_name):
     print(vehicles[column_name].value_counts(dropna=False),'\n')
     print(vehicles[column_name].describe())
     print(vehicles[vehicles[column_name].isna()])

In [ ]:
quick_column('model_year')
#Oldest Cars: 1908, Newest: 2019
#Median: 2011, Mean: ≈2009

# 3619/51525 missing values (≈7% of rows have NaN)
#No noticible coorelation between contents of rows with missing values in this column
#vehicles['model_year']=vehicles['model_year'].astype('Int64')

In [ ]:
quick_column('cylinders')
# 5260/51525 missing values (≈10% of rows have NaN)
#No noticible coorelation between contents of rows with missing values in this column

#Least Cylinders: 3, Most Cylinders: 12, Median: 6, Mean: ≈6
#vehicles['model_year']=vehicles['model_year'].astype('Int64')

In [ ]:
quick_column('odometer')
#7892/51525 missing values (≈15% of rows have NaN)
#No noticible coorelation between contents of rows with missing values in this column

#Min: 0, Max:990000, Median: 113000, Mean: ≈115553

In [ ]:
fig1 = px.histogram(vehicles, x = 'odometer',nbins=100, labels={'Milage':'Number of Cars'})
fig1.show()

In [ ]:
quick_column('paint_color')
#9267/51525 missing values (≈18 of rows have NaN)

#Top Colors(Most to least frequent): 1.White, 2.Black, 3.Silver, 4.Grey, 5.Blue, 6.Red, 7.Green, 8.Brown

In [ ]:
quick_column('is_4wd')
#Nearly Half of rows is NaN, while other half is 1.0. Based on this we can conclude 1.0 means Yes, there is 4wheeldrive('is_4wd'),
# thus NaN means No. Thus I will convert NaN to the value 0.0 so that I can then convert column to int64,
#where 1 means yes, 0 means no.
vehicles['is_4wd'] = vehicles['is_4wd'].fillna(0).astype('int64')

In [ ]:
#convert 'date_posted' column to datetime64
vehicles['date_posted'] = pd.to_datetime(vehicles['date_posted'], format='%Y-%m-%d')
print(vehicles['date_posted'].head())

In [ ]:
fig1 = px.histogram(
    vehicles,
    x='price',
    nbins=1000,
    title="Distribution of Vehicle Prices",
    labels={"price": "Price of Vehicles(in USD)"}
)
fig1.show()
#st.plotly_chart(fig1, use_container_width=True) #auto resizes chart

In [ ]:
fig1 = px.histogram(
    vehicles,
    x='odometer',
    nbins=1000,
    title="Distribution of Vehicle Mileage",
    labels={"odometer": "Mileage"}
    
)
fig1.update_xaxes(tickformat=",")
fig1.show()
#st.plotly_chart(fig1, use_container_width=True) #auto resizes chart

In [ ]:
quality_order=['salvage', 'fair','good','excellent', 'like new', 'new'] 

# make condition ordered categorical
vehicles['condition'] = pd.Categorical(
    vehicles['condition'],
    categories=quality_order,
    ordered=True
)
fig2 = px.scatter(
    vehicles, 
    x='odometer', 
    y='price', 
    color='condition',
    opacity=0.3,
    category_orders={'condition': quality_order},
    title="Price vs Mileage Scatter Plot",
    labels={"odometer": "Mileage", "price": "Price of Vehicles(in USD)"})
fig2.update_xaxes(tickformat=",")
fig2.show()
#st.plotly_chart(fig2, use_container_width=True) #auto resizes chart

In [ ]:
#conditions in order(worst to best)
quality_order=['salvage', 'fair','good','excellent', 'like new', 'new'] 

# make condition ordered categorical
vehicles['condition'] = pd.Categorical(
    vehicles['condition'],
    categories=quality_order,
    ordered=True
)
#create a 2 column dataframe based on condition and count
condition_df = vehicles['condition'].value_counts().sort_index().reset_index()
condition_df.columns = ['condition', 'count']
print(condition_df)

#reverse_condition = st.checkbox("Reverse Order of Condition (best -> worst)")
#order = quality_order[::-1] if reverse_condition else quality_order #if check box activated reverse order
fig3 = px.bar(
    condition_df, 
    category_orders={'condition': order}, 
    x='condition',
    y='count', 
    labels={"condition": "Condition","count": "Number of Vehicles"}, 
    title="Vehicle Condition")
fig3.show()
#st.write("Number of vehicles found in each condition")
#st.plotly_chart(fig3, use_container_width=True #auto resizes chart

We can see that the majority of cars are in good or excellent condition. 